In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import gensim

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
data_root = '/home/thenhz/workspace/hate-speech/data/'

GLOVE_MODEL_FILE = data_root + 'word2vec/glove_WIKI'

data = pd.read_csv('data/hate-speech/haspeede_TW-train.tsv', sep='\t', names=['text','sentiment'])
print("data file loaded..")
word2vec_model = gensim.models.Word2Vec.load(GLOVE_MODEL_FILE)
print("word2vec model loaded...")



data file loaded..
word2vec model loaded...


In [3]:
#data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 0].size) #positive
print(data[ data['sentiment'] == 1].size) #negative



4052
1944


In [4]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [5]:
len(tokenizer.word_index)

12011

In [13]:
word_vectors = word2vec_model.wv
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200


In [14]:
len(word_vectors.vocab)

730613

In [15]:
WV_DIM = 100

# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(max_fatures, WV_DIM) - 0.5) / 5.0
for word, i in tokenizer.word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass 

In [16]:
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, WV_DIM,weights=[wv_matrix], input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

#model = Sequential()
#model.add(LSTM(32, return_sequences=True,
#               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32))  # return a single vector of dimension 32
#model.add(Dense(10, activation='softmax'))

#model.compile(loss='categorical_crossentropy',
#              optimizer='rmsprop',
#              metrics=['accuracy'])

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 100)           200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               232848    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 433,242
Trainable params: 433,242
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2008, 30) (2008, 2)
(990, 30) (990, 2)


In [18]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 10, validation_split = 0.1, batch_size=batch_size, verbose = 2)

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1807 samples, validate on 201 samples
Epoch 1/10
 - 4s - loss: 0.5968 - acc: 0.6879 - val_loss: 0.5221 - val_acc: 0.7264
Epoch 2/10
 - 3s - loss: 0.4310 - acc: 0.8041 - val_loss: 0.4814 - val_acc: 0.7363
Epoch 3/10
 - 3s - loss: 0.3424 - acc: 0.8517 - val_loss: 0.4735 - val_acc: 0.7811
Epoch 4/10
 - 3s - loss: 0.2922 - acc: 0.8854 - val_loss: 0.5825 - val_acc: 0.7562
Epoch 5/10
 - 3s - loss: 0.2472 - acc: 0.8943 - val_loss: 0.6033 - val_acc: 0.7662
Epoch 6/10
 - 3s - loss: 0.2175 - acc: 0.9175 - val_loss: 0.5835 - val_acc: 0.7463
Epoch 7/10
 - 3s - loss: 0.1667 - acc: 0.9369 - val_loss: 0.6751 - val_acc: 0.7861
Epoch 8/10
 - 3s - loss: 0.1560 - acc: 0.9408 - val_loss: 0.6586 - val_acc: 0.7512
Epoch 9/10
 - 3s - loss: 0.1258 - acc: 0.9530 - val_loss: 0.7925 - val_acc: 0.7811
Epoch 10/10
 - 3s - loss: 0.1003 - acc: 0.9707 - val_loss: 0.8245 - val_acc: 0.7662


In [20]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score = model.evaluate(X_test, Y_test, batch_size = batch_size)
#logger.info('mse=%f, mae=%f, mape=%f' % (scores[0],scores[1],scores[2]))
#print(score)
#print("score: %.2f " % (score))
#print("acc: %.2f" % (acc))
print(score)

[]


In [21]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 55.5921052631579 %
neg_acc 84.54810495626822 %
